In [622]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LinearRegression

In [623]:
train = pd.read_csv('train.csv')
train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']
test = pd.read_csv('test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [624]:
train_x.shape

(891, 11)

In [625]:
def gender(x):
    if x == 'male':
        return 1
    else:
        return 0
    
def cabin(x):
    if pd.isna(x):
        return 0
    else:
        return 1
    
def rich(x):
    if x > 30:
        return 1
    else:
        return 0

def name(x):
    if "Miss" in x:
        return 1
    elif "Mr." in x:
        return 2
    elif "Master." in x:
        return 3
    elif "Mrs." in x:
        return 4
    else:
        return 0

In [626]:
train_x.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [627]:
train_x['Embarked'].fillna('S', inplace=True)
test['Embarked'].fillna('S', inplace=True)

In [628]:
embarked_map = {'S':1, 'C':2, 'Q':3}

In [629]:
train_x['Embarked'] = train_x['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].map(embarked_map)

In [630]:
train_x['Title'] = train_x['Name'].apply(lambda x: name(x))
train_x['Name'] = train_x['Name'].apply(lambda x: len(x))
train_x['Sex'] = train_x['Sex'].apply(lambda x: gender(x))
train_x['family'] = train_x['SibSp'] + train_x['Parch']
train_x['Cabin'] = train_x['Cabin'].apply(lambda x: cabin(x))
train_x['rich'] = train_x['Fare'].apply(lambda x: rich(x))

In [631]:
test['Title'] = test['Name'].apply(lambda x: name(x))
test['Name'] = test['Name'].apply(lambda x: len(x))
test['Sex'] = test['Sex'].apply(lambda x: gender(x))
test['family'] = test['SibSp'] + train_x['Parch']
test['Cabin'] = test['Cabin'].apply(lambda x: cabin(x))
test['rich'] = test['Fare'].apply(lambda x: rich(x))

In [632]:
missAge = train_x[train_x['Age'].isna()]
haveAge = train_x[train_x['Age'].notna()]
testAge = test[test['Age'].isna()]
testHaveAge = test[test['Age'].notna()]

In [633]:
res_age = haveAge[['Title','Parch']]
test_age = haveAge['Age']
final_age = missAge[['Title','Parch']]
sub_age = missAge[['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']]
test_sub_age = testAge[['PassengerId','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']]
test_final_age = testAge[['Title','Parch']]

In [634]:
linR = LinearRegression()
linR.fit(res_age,test_age)
sub_age['Age'] = linR.predict(final_age)
test_sub_age['Age'] = linR.predict(test_final_age)

C:\Users\Edison\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Edison\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [635]:
filled_data = haveAge.append(sub_age)
filled_test_data = testHaveAge.append(test_sub_age)

C:\Users\Edison\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [636]:
submit = pd.DataFrame()
submit['PassengerId'] = test['PassengerId']

In [637]:
filled_data = filled_data.drop(['Ticket','PassengerId'], axis=1)
filled_test_data = filled_test_data.drop(['Ticket', 'PassengerId'], axis=1)

In [638]:
filled_data.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [639]:
corr_df = filled_data.copy()
corr_df['Res'] = train_y
corr = corr_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Title,family,rich,Res
Age,1,0.233109,0.0146217,0.0877602,0.0330956,-0.20129,-0.334984,0.0922073,-0.265456,0.0494247,-0.295825,0.0996335,-0.0666428
Cabin,0.233109,1,0.0137744,0.482075,0.190943,0.036987,-0.725541,-0.140391,-0.04046,0.0978857,0.0342359,0.557389,0.316912
Embarked,0.0146217,0.0137744,1,0.0621416,-0.107749,-0.0786647,0.0457018,-0.116569,-0.0599614,-0.260153,-0.0903666,0.0153204,0.106811
Fare,0.0877602,0.482075,0.0621416,1,0.155832,0.216225,-0.5495,-0.182333,0.159651,0.0643507,0.215135,0.605056,0.257307
Name,0.0330956,0.190943,-0.107749,0.155832,1,0.252282,-0.220001,-0.448759,0.165019,0.426877,0.283103,0.223859,0.33235
Parch,-0.20129,0.036987,-0.0786647,0.216225,0.252282,1,0.0184427,-0.245489,0.414838,0.238173,0.777299,0.280789,0.0816294
Pclass,-0.334984,-0.725541,0.0457018,-0.5495,-0.220001,0.0184427,1,0.1319,0.0830814,-0.0995771,0.00192293,-0.578504,-0.338481
Sex,0.0922073,-0.140391,-0.116569,-0.182333,-0.448759,-0.245489,0.1319,1,-0.114631,-0.107355,-0.194921,-0.159412,-0.543351
SibSp,-0.265456,-0.04046,-0.0599614,0.159651,0.165019,0.414838,0.0830814,-0.114631,1,0.0708996,0.610011,0.213239,-0.0353225
Title,0.0494247,0.0978857,-0.260153,0.0643507,0.426877,0.238173,-0.0995771,-0.107355,0.0708996,1,0.342156,0.193864,0.138438


In [640]:
#filled_data = filled_data.drop(['Name','Fare','rich'],axis = 1)
#test = test.drop(['Name','Fare','rich'],axis = 1)
#filled_data = filled_data.drop('PassengerId',axis=1)

In [641]:
filled_data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Title,family,rich
0,22.0,0,1,7.2500,23,0,3,1,1,2.0,1.0,0.0
1,38.0,1,2,71.2833,51,0,1,0,1,4.0,1.0,1.0
2,26.0,0,1,7.9250,22,0,3,0,0,1.0,0.0,0.0
3,35.0,1,1,53.1000,44,0,1,0,1,4.0,1.0,1.0
4,35.0,0,1,8.0500,24,0,3,1,0,2.0,0.0,0.0


In [642]:
x_train, x_test, y_train, y_test = train_test_split(filled_data,train_y)

In [643]:
clf = GradientBoostingClassifier()
clf.fit(x_train, y_train)
prediction = clf.predict(x_test)
accuracy_score(prediction, y_test)

0.5515695067264574

In [644]:
lr = LogisticRegression(solver='newton-cg')
lr.fit(x_train,y_train)
prediction = lr.predict(x_test)
accuracy_score(prediction,y_test)

0.5919282511210763

In [645]:
rf = RandomForestClassifier(n_estimators = 5)
rf.fit(x_train,y_train)
prediction = rf.predict(x_test)
accuracy_score(prediction,y_test)

0.5022421524663677

In [646]:
estimators = [('clf',clf),('lr',lr),('rf',rf)]
ensemble = VotingClassifier(estimators, voting = 'hard')
ensemble.fit(x_train,y_train)
prediction = ensemble.predict(x_test)
accuracy_score(prediction,y_test)

0.5605381165919282

In [357]:
submit['Survived'] = ensemble.predict(test)
submit.head()


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [358]:
submit.to_csv('submission.csv')